# Installing and importing dependencies

In [1]:
# !pip install tensorflow==2.5.1 tensorflow-gpu==2.5.1 opencv-python mediapipe sklearn matplotlib

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Collecting keypoints from mediapipe holistics

In [3]:
mp_holistic = mp.solutions.holistic  #we are bringing the holistic model of mideapipe
mp_drawing = mp.solutions.drawing_utils #we are also bringing the drawing utilities of mideapipe.

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(70,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(70,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(70,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(70,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# Extracting keypoints value

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# Folder setup for data Collection

In [7]:
#path for exported data,numpy arrays
PATH = os.path.join("MP_Data")

#Actions that we are trying to detect
# 
actions =(['hello','goodbye','please','no','thanks','yes'])



#total number of sequences that we are trying to collect which is 30 videos worth of data
no_sequences = 30

#30 frames length of video
sequence_length = 30

#folder start
start_folder = 30

# COLLECTING KEYPOINTS FOR TRAINING AND TESTING

# Preprocessing our data

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'hello': 0, 'goodbye': 1, 'please': 2, 'no': 3, 'thanks': 4, 'yes': 5}

In [10]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        video = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(PATH, action, str(sequence), "{}.npy".format(frame_num)))
            video.append(res)
        sequences.append(video)
        labels.append(label_map[action])

In [12]:
X = np.array(sequences)
X.shape

(180, 30, 1662)

In [13]:
y = to_categorical(labels).astype(int)
y

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
X_train.shape

(171, 30, 1662)

# Building and training LSTM DL Model

# Predicting result from test data

In [15]:
from tensorflow.keras.models import load_model
model = load_model(r'C:\Users\KIIT\Desktop\project_video_chat\8thSemProject_part2\8thSemProject\action.h5')

In [16]:
result = model.predict(X_test)

1/1 [==============================] - 3s 3s/step


In [17]:
actions[np.argmax(result[1])]

'no'

In [18]:
actions[np.argmax(y_test[1])]

'no'

# Saving weights for future

# Accuracy Calculation and Evaluation using Confusion matrix

In [19]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [20]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 75ms/step


In [21]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [22]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[7, 0],
        [0, 2]],

       [[8, 0],
        [0, 1]],

       [[5, 0],
        [0, 4]],

       [[7, 0],
        [0, 2]]], dtype=int64)

In [23]:
accuracy_score(ytrue, yhat)

1.0

# Real Time Detection

In [3]:
sequence = []
sentence = []
predictions = []
threshold = 0.7

cap = cv2.VideoCapture(0)
# Set mediapipe model 
while cap.isOpened():
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            # print(actions[np.argmax(res)])
            # print('-----------------------------------------------')
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]
                
            cv2.rectangle(image, (0,0), (640, 60), (0, 102, 255), -1)
            cv2.putText(image, ' '.join(sentence), (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        print(sentence[-1])
        # print(frame)
        # Break gracefully
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined